In [1]:
import torchvision.transforms as transforms
import torch

from src.concept_bottleneck.dataset import CUB200ImageToAttributes

# According to: https://pytorch.org/hub/pytorch_vision_inception_v3/
preprocess = transforms.Compose(
    [
        transforms.Resize(299),
        transforms.CenterCrop(299),
        transforms.ToTensor(),
        transforms.Normalize(
            mean=[0.485, 0.456, 0.406],
            std=[0.229, 0.224, 0.225],
        ),
    ]
)

training_data: CUB200ImageToAttributes[
    torch.Tensor, torch.Tensor
] = CUB200ImageToAttributes(
    train=True, transform=preprocess, target_transform=torch.from_numpy  # type: ignore
)
test_data: CUB200ImageToAttributes[
    torch.Tensor, torch.Tensor
] = CUB200ImageToAttributes(
    train=False, transform=preprocess, target_transform=torch.from_numpy  # type: ignore
)


Using downloaded and verified file: /home/shuangwu/interactive-concept-bottleneck/src/concept_bottleneck/data/CUB_200_2011.tgz
Extracting /home/shuangwu/interactive-concept-bottleneck/src/concept_bottleneck/data/CUB_200_2011.tgz to /home/shuangwu/interactive-concept-bottleneck/src/concept_bottleneck/data


In [2]:
from torch.utils.data import DataLoader

batch_size = 16
num_workers = 2

training_dataloader = DataLoader(
    training_data, batch_size=batch_size, shuffle=True, num_workers=num_workers
)
test_dataloader = DataLoader(test_data, batch_size=batch_size, num_workers=num_workers)


In [3]:
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using {device} device")

model: torch.nn.Module = torch.hub.load(
    "pytorch/vision:v0.10.0",
    "inception_v3",
    init_weights=False,
    num_classes=training_data.num_attributes,
)
model = model.to(device)


Using cuda device


Using cache found in /home/shuangwu/.cache/torch/hub/pytorch_vision_v0.10.0


In [4]:
def train(
    dataloader: DataLoader[tuple[torch.Tensor, torch.Tensor]],
    model: torch.nn.Module,
    loss_fn: torch.nn.Module,
    optimizer: torch.optim.Optimizer,
):
    size = len(dataloader.dataset)  # type: ignore
    model.train()
    for batch, (X, y) in enumerate(dataloader):
        X, y = X.to(device), y.to(torch.float).to(device)  # type: ignore

        logits, aux_logits = model(X)
        loss = loss_fn(logits, y) + 0.4 * loss_fn(aux_logits, y)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if batch % 100 == 0:
            loss, current = loss.item(), batch * len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")


In [5]:
def test(
    dataloader: DataLoader[tuple[torch.Tensor, torch.Tensor]],
    model: torch.nn.Module,
    loss_fn: torch.nn.Module,
):
    size = len(dataloader.dataset)  # type: ignore
    num_batches = len(dataloader)
    model.eval()
    test_loss, correct = 0, 0
    with torch.no_grad():
        sigmoid = torch.nn.Sigmoid()
        for X, y in dataloader:
            X, y = X.to(device), y.to(device)  # type: ignore
            logits = model(X)
            test_loss += loss_fn(logits, y.to(torch.float)).item()
            correct_attributes = torch.sum(
                ((sigmoid(logits) > 0.5).to(torch.int64) == y).to(torch.float)
            )
            num_attributes = y.shape[1]
            correct += correct_attributes.item() / num_attributes
    test_loss /= num_batches
    accuracy = correct / size
    return test_loss, accuracy


In [6]:
def save_model(model: torch.nn.Module, filename: str):
    torch.save(model.state_dict(), f"{filename}.pth")
    print(f"Saved PyTorch Model State to {filename}.pth")


def load_model(model: torch.nn.Module, filename: str):
    model.load_state_dict(torch.load(f"{filename}.pth"))
    print(f"Loaded PyTorch Model State from {filename}.pth")
    model.eval()


In [7]:
def run_epoch(epochs: int):
    loss_fn = torch.nn.BCEWithLogitsLoss()
    optimizer = torch.optim.SGD(model.parameters(), lr=1e-3)

    train_losses: list[float] = []
    train_accuracies: list[float] = []
    test_losses: list[float] = []
    test_accuracies: list[float] = []
    for t in range(epochs):
        print(f"\nEpoch {t+1}\n-------------------------------")
        train(training_dataloader, model, loss_fn, optimizer)
        train_loss, train_accuracy = test(training_dataloader, model, loss_fn)
        print(
            f"Train Accuracy: {(100 * train_accuracy):>0.10f}%, Avg loss: {train_loss:>8f}"
        )
        train_losses.append(train_loss)
        train_accuracies.append(train_accuracy)

        test_loss, test_accuracy = test(test_dataloader, model, loss_fn)
        print(
            f"Test Accuracy: {(100 * test_accuracy):>0.10f}%, Avg loss: {test_loss:>8f}"
        )
        test_losses.append(test_loss)
        test_accuracies.append(test_accuracy)

        if test_accuracy > 0.85 and t % 5 == 0:
            save_model(model, f"inception_model_{t}")

        if test_accuracy > 0.98:
            print("Reached 98% accuracy so cancelling training")
            break

    print("Done!")
    return train_losses, train_accuracies, test_losses, test_accuracies


In [8]:
train_losses, train_accuracies, test_losses, test_accuracies = run_epoch(epochs=100)
save_model(model, "inception_model_final")

import json

with open("train_losses.json", "w") as f:
    json.dump(train_losses, f)

with open("train_accuracies.json", "w") as f:
    json.dump(train_accuracies, f)

with open("test_losses.json", "w") as f:
    json.dump(test_losses, f)

with open("test_accuracies.json", "w") as f:
    json.dump(test_accuracies, f)



Epoch 1
-------------------------------
loss: 0.995102  [    0/ 5994]
loss: 0.979747  [ 1600/ 5994]
loss: 0.963969  [ 3200/ 5994]
loss: 0.951391  [ 4800/ 5994]
Train Accuracy: 66.6662388885%, Avg loss: 0.652709
Test Accuracy: 66.6851428976%, Avg loss: 0.652778

Epoch 2
-------------------------------
loss: 0.935878  [    0/ 5994]
loss: 0.920274  [ 1600/ 5994]
loss: 0.915019  [ 3200/ 5994]
loss: 0.891778  [ 4800/ 5994]
Train Accuracy: 80.0644661756%, Avg loss: 0.600265
Test Accuracy: 80.0677978103%, Avg loss: 0.600964

Epoch 3
-------------------------------
loss: 0.883202  [    0/ 5994]
loss: 0.879162  [ 1600/ 5994]
loss: 0.857617  [ 3200/ 5994]
loss: 0.850683  [ 4800/ 5994]
Train Accuracy: 86.9950613006%, Avg loss: 0.552515
Test Accuracy: 86.9904653797%, Avg loss: 0.553529

Epoch 4
-------------------------------
loss: 0.844376  [    0/ 5994]
loss: 0.831990  [ 1600/ 5994]
loss: 0.809481  [ 3200/ 5994]
loss: 0.806807  [ 4800/ 5994]
Train Accuracy: 89.4516311183%, Avg loss: 0.515392
Te

KeyboardInterrupt: 